In [1]:
from collections import deque
from functools import reduce
from operator import xor

In [2]:
def knot(l, lengths, i, skip):
    n = len(l)
    d = deque(l)
    for length in lengths:
        d.rotate(-i)
        l = list(d)
        l[:length] = reversed(l[:length])
        d = deque(l)
        d.rotate(i)
        i = (i + length + skip) % n
        skip += 1
    return i, skip, list(d)

def get_ascii(input):
    output = [ord(s) for s in str(input)]
    output += [17, 31, 73, 47, 23]
    return output

def get_sparse_hash(l, lengths):
    i = 0
    skip = 0
    for _ in range(64):
        i, skip, l = knot(l, lengths, i, skip)
    return l

def get_dense_hash(sparse_hash):
    dense_hash = []
    for i in range(0, len(sparse_hash), 16):
        block = sparse_hash[i:i + 16]
        dense_hash.append(reduce(xor, block))
    return dense_hash

def get_binary_hash(knot_hash):
    return ''.join([f"{int(x, 16):04b}" for x in knot_hash])

def get_disk(key):
    disk = []
    for i in range(128):
        l = list(range(256))
        lengths = get_ascii(key + '-' + str(i))
        sparse_hash = get_sparse_hash(l, lengths)
        dense_hash = get_dense_hash(sparse_hash)
        knot_hash = ''.join([f'{x:02x}' for x in dense_hash])
        bin_hash = get_binary_hash(knot_hash)
        disk.append([int(x) for x in bin_hash])
    return disk


In [3]:
key = 'stpzcrnm'
disk = get_disk(key)

In [4]:
sum(sum(x) for x in disk)

8250

In [5]:
def find_regions(disk):
    region = {}
    new_reg = 1
    for i in range(128):
        for j in range(128):
            if disk[i][j] == 1 and (i, j) not in region:
                region[i, j] = new_reg
                q = deque([(i, j)])
                while q:
                    vi, vj = q.popleft()
                    if disk[vi][vj] == 0:
                        continue
                    for (di, dj) in [(-1, 0), (1, 0), (0, -1), (0, 1)]:
                        ni, nj = vi + di, vj + dj
                        if ni >= 0 and ni < 128 and nj >= 0 and nj < 128 and disk[ni][nj] == 1:
                            if (ni, nj) not in region:
                                region[ni, nj] = new_reg
                                q.append((ni, nj))
                new_reg += 1
    return region


In [6]:
regions = find_regions(disk)

In [7]:
max(regions.values())

1113